In [1]:
from torchHMM.model.discretized_HMM import DiscreteHMM, DISCRETIZATION_TECHNIQUES
from torchHMM.model.discretized_flow_HMM import FlowHMM

In [2]:
import numpy as np
import pandas as pd
import pickle as pkl

In [3]:
from hmmlearn.hmm import CategoricalHMM, GaussianHMM
from gensim.models import KeyedVectors


In [4]:
with open('clickstream_experiment/data/preprocessed_data/ClickStream_test_cleaned.pkl', 'rb') as f:
    cs = pkl.load(f)
session_train, sessions_test = cs.sessions[:100000], cs.sessions[100000:110000]

In [5]:
item_ids = np.unique([e.item_id for s in session_train for e in s.event_list])
item_ids = {item_ids[i]: i for i in range(len(item_ids))}

Xs = [[item_ids[e.item_id] for e in s.event_list] for s in session_train]
l = [len(y) for y in Xs]

X = np.concatenate(Xs).reshape(-1, 1)

target =  [[item_ids[e.item_id] for e in s.event_list if e.item_id in item_ids.keys()] for s in sessions_test]
lt = np.array([len(y) for y in target])
target = np.concatenate(target).reshape(-1, 1)

In [6]:
chmm = CategoricalHMM(5, n_iter=10, verbose=True)


In [7]:
%time chmm.fit(X, l)

         1   -10919453.8211             +nan
         2    -9992095.9652     +927357.8560
         3    -9896268.8141      +95827.1511
         4    -9830764.7020      +65504.1121
         5    -9781417.6486      +49347.0533
         6    -9743646.9014      +37770.7473
         7    -9716188.3315      +27458.5699
         8    -9696714.5646      +19473.7669
         9    -9682445.7468      +14268.8178


CPU times: user 2min 12s, sys: 31.8 ms, total: 2min 12s
Wall time: 2min 12s


        10    -9671265.8355      +11179.9113


CategoricalHMM(n_components=5,
               random_state=RandomState(MT19937) at 0x7F2A17DF8A40,
               verbose=True)

In [8]:
print(chmm.score(target, lt))

IndexError: arrays used as indices must be of integer (or boolean) type

In [9]:
vectors = KeyedVectors.load(
    f"clickstream_experiment/data/preprocessed_data/vectors_train_20_10_5_8_cleaned.kv"
)
vecs = np.concatenate(
    [
        vectors.get_vector(k).reshape(1, -1)
        for k in list(vectors.key_to_index.keys())
    ]
)

In [10]:
unknown = vecs.mean(axis=0).reshape(1, -1)

def get_vec(e):
    try:
        return vectors.get_vector(e.item_id).reshape(1, -1)
    except:
        return unknown

In [11]:
Xs_c = [[get_vec(e) for e in s.event_list] for s in session_train]
l_c = np.array([len(y) for y in X])

X_c = np.concatenate([a for as_ in Xs_c for a in as_], axis=0)

target_c =  [[get_vec(e) for e in s.event_list if e.item_id in item_ids.keys()] for s in sessions_test]
target_c = np.concatenate([a for as_ in target_c for a in as_], axis=0)


In [12]:
w2vhmm = GaussianHMM(5, n_iter=10, verbose=True)

In [13]:
%time w2vhmm.fit(X_c, l_c)

/ziob/klaudia/miniconda3/envs/preproc/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
         1    13671830.3651             +nan
         2    40479413.7090   +26807583.3439
         3    66813156.8904   +26333743.1813
         4   142205733.2103   +75392576.3200
         5   142311758.6624     +106025.4521
         6   142345166.3955      +33407.7330
         7   142353988.7753       +8822.3799
         8   142357606.7115       +3617.9362
         9   142358997.8106       +1391.0991
        10   142359428.1960        +430.3853
Some rows of transmat_ have zero sum because no transition from the state was ever observed.


CPU times: user 18min 30s, sys: 2.57 s, total: 18min 33s
Wall time: 18min 27s


GaussianHMM(n_components=5, verbose=True)

In [14]:
print(w2vhmm.score(target_c, lt))

ValueError: transmat_ rows must sum to 1 (got [0. 0. 0. 0. 0.])

In [15]:
w2v_dhmm = DiscreteHMM(n_components=5, no_nodes=512, l=100, optimizer="Adam", verbose=True)

In [16]:
%time w2v_dhmm.fit(X_c, l_c)

/ziob/klaudia/miniconda3/envs/preproc/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/ziob/klaudia/FlowHMM/torchHMM/model/discretized_HMM.py:447: RuntimeWarning: invalid value encountered in divide
  cooc_matrix /= cooc_matrix.sum()


ValueError: component 0 of 'full' covars must be symmetric, positive-definite

In [17]:
print(w2v_dhmm.score(target_c, lt))

ValueError: array must not contain infs or NaNs

In [18]:
w2v_fhmm = FlowHMM(n_components=5, no_nodes=512, l=100, optimizer="Adam", verbose=True)

%time w2v_fhmm.fit(X_c, l_c)

print(w2v_fhmm.score(target_c, lt))


/ziob/klaudia/FlowHMM/torchHMM/model/discretized_flow_HMM.py:417: RuntimeWarning: invalid value encountered in divide
  cooc_matrix /= cooc_matrix.sum()


RuntimeError: Index put requires the source and destination dtypes match, got Double for the destination and Float for the source.

TypeError: FlowHMM.score() takes 2 positional arguments but 3 were given